BVP

laplace equation
ball around center (0, 0) - drichlet bc 
smallest eigenvalues and functions

In [1]:
import numpy as np
import ufl
from mpi4py import MPI
from petsc4py import PETSc
from slepc4py import SLEPc
import pyvista as pv
from dolfinx import mesh, fem, plot

#1 generate a ball domain 

In [2]:
sphere_radius = 1.0

In [3]:
import gmsh
from dolfinx.io import gmshio
# import meshio


def get_sphere_domain(radius: float, res :float = 1, filename: str = None):
    """
    radius: float. Radius of sphere
    res: float, resolution of mesh
    """

    # if filename is not None:
        
        


    gmsh.initialize()
    model = gmsh.model()
    model.add("Sphere")
    model.setCurrent("Sphere")


    sphere = model.occ.addSphere(0, 0, 0, radius=radius, tag=1)
    model.occ.synchronize()
    gdim = 3
    model.addPhysicalGroup(dim=gdim, tags=[sphere])


    #set mesh options and generate mesh
    gmsh.option.setNumber("Mesh.MeshSizeFactor", res)
    
    model.mesh.generate(gdim)

    #get domain from mesh
    domain, _, _ = gmshio.model_to_mesh(model=model, comm=MPI.COMM_WORLD, rank = 0, gdim = gdim)

    
    
    # gmsh.write("Sphere.xmdf")

    gmsh.finalize()
    return domain

In [4]:
from dolfinx import plot
import pyvista

def plot_domain(domain):

    pyvista.start_xvfb()
    gdim = domain.topology.dim #dimensions of domain!

    domain.topology.create_connectivity(gdim, gdim)
    topology, cell_types, geometry = plot.vtk_mesh(domain, gdim)
    grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, show_edges=True)
    plotter.view_xy()
    if not pyvista.OFF_SCREEN:
        plotter.show()
    else:
        figure = plotter.screenshot("fundamentals_mesh.png")



In [5]:
domain = get_sphere_domain(sphere_radius)
plot_domain(domain)

Info    : Meshing 1D...
Info    : [ 40%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.000755006s, CPU 0.001083s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Sphere, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0126254s, CPU 0.010698s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 162 nodes...
Info    : Done tetrahedrizing 170 nodes (Wall 0.00186512s, CPU 0.000684s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.00385719s, CPU 0.002471s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 3.44752 (nodes removed 0 0)
Info    : 3D refinement terminated (205 nodes total):
Info    :  - 0 Delaunay cavities modified for star shapeness
Info    :  - 0 nodes could not be inserted
Info    :  - 692 tetrahedra created in 0.00184878 sec. (374301 tets/

Widget(value='<iframe src="http://localhost:44609/index.html?ui=P_0x7b2bc65ff770_0&reconnect=auto" class="pyvi…

#2 Setup function space and apply boundary conditions

In [6]:
def setup_fe_space(domain):
    """Create function space and variational forms. Defining bilinear and linear forms."""
    V = fem.functionspace(domain, ("Lagrange", 1))

    u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
    a = fem.form(ufl.inner(ufl.grad(u), ufl.grad(v))*ufl.dx)
    m = fem.form(ufl.inner(u, v)*ufl.dx)

    return V, a, m


In [7]:
V, a, m = setup_fe_space(domain)

preparing boundary conditions

In [8]:
from petsc4py import PETSc
def on_boundary(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2 + x[2]**2), sphere_radius)

def get_bcs(V):
    boundary_dofs = fem.locate_dofs_geometrical(V, on_boundary)
    bc = fem.dirichletbc(PETSc.ScalarType(0), boundary_dofs, V)

    return bc


#3 assemble matrices and apply boundary conditions

In [9]:
import dolfinx.fem.petsc

def assemble_and_apply_bc(V, a, m):

    #get bcs

    bc = get_bcs(V)

    #  Assemble matrices A and M

    A = fem.petsc.assemble_matrix(a, bcs=[]) #bc only on stiffness matrix
    A.assemble()
    M = fem.petsc.assemble_matrix(m) # mass matrix
    M.assemble()

    # null_vec = A.createVecLeft()   #for neumann problem
    # null_vec.set(1.0)
    # null_vec.normalize()
    # nullspace = PETSc.NullSpace().create(vectors=[null_vec])
    # A.setNullSpace(nullspace)

    return A, M



In [10]:
A, M = assemble_and_apply_bc(V, a, m)

setup eigenvalue solver

In [11]:

def get_eigenvalues(A, M, n_eigen):

    eigensolver = SLEPc.EPS().create()
    eigensolver.setOperators(A, M)
    eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    # eigensolver.setDimensions(nev=n_eigen, ncv=2*n_eigen)  # Larger subspace
    eigensolver.setDimensions(nev=n_eigen)  # Larger subspace
    eigensolver.setTolerances(tol=1e-8, max_it=1000)
    eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_MAGNITUDE)

    eigensolver.setFromOptions()
    eigensolver.solve()
    n_conv = eigensolver.getConverged()

    if MPI.COMM_WORLD.rank == 0:
        print(f"Number of converged eigenvalues: {n_conv}")


    return eigensolver, n_conv

In [12]:
n_eigen = 5
eigensolver, n_conv = get_eigenvalues(A, M, n_eigen=n_eigen)

Number of converged eigenvalues: 9


In [13]:
import matplotlib.pyplot as plt
import numpy as np
import dolfinx.plot

def plot_eigenfunctions(domain, V, eigensolver, A, nev_to_plot):
    """
    Plot the first few eigenfunctions for any given domain using matplotlib.
    Parameters:
        domain: dolfinx.mesh.Mesh
        V: dolfinx.fem.FunctionSpace
        eigensolver: SLEPc.EPS
        A: PETSc.Mat (needed for getVecs)
        n_conv: int (number of converged eigenpairs)
        nev_to_plot: int (number of eigenfunctions to plot)
    """
    if nev_to_plot == 0:
        return
    
    if MPI.COMM_WORLD.rank != 0:
        return

    topology, cell_types, geometry = plot.vtk_mesh(V)
    points = domain.geometry.x
    for i in range(nev_to_plot):
        eigval = eigensolver.getEigenvalue(i)
        print(f"Eigenvalue {i}: {eigval:.4f}")

        r, _ = A.getVecs()
        eigensolver.getEigenvector(i, r)

        uh = fem.Function(V)
        uh.x.petsc_vec.setArray(r.array)
        uh.x.petsc_vec.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

        # Create PyVista grid
        values = uh.x.array.real
        grid = pv.UnstructuredGrid(topology, cell_types, points)
        grid.point_data["u"] = values

        # Plot
        plotter = pv.Plotter(notebook=True)
        plotter.add_mesh(grid, show_edges=True, scalars="u", cmap="hsv")
        plotter.view_xy()
        plotter.add_text(f"Eigenfunction {i}, λ = {eigval:.4f}", font_size=12)
        plotter.show()

# Usage example (after solving eigenproblem):
# plot_eigenfunctions(domain, V, eigensolver, A, n_conv, nev_to_plot=6)


In [14]:
plot_eigenfunctions(domain, V, eigensolver, A, min(n_eigen, n_conv))

Eigenvalue 0: 0.0000


Widget(value='<iframe src="http://localhost:44609/index.html?ui=P_0x7b2bb1e95bd0_1&reconnect=auto" class="pyvi…

Eigenvalue 1: 4.5802


Widget(value='<iframe src="http://localhost:44609/index.html?ui=P_0x7b2bb1e96e90_2&reconnect=auto" class="pyvi…

Eigenvalue 2: 4.5912


Widget(value='<iframe src="http://localhost:44609/index.html?ui=P_0x7b2bb1e96d50_3&reconnect=auto" class="pyvi…

Eigenvalue 3: 4.6065


Widget(value='<iframe src="http://localhost:44609/index.html?ui=P_0x7b2bb0a0c050_4&reconnect=auto" class="pyvi…

Eigenvalue 4: 12.4604


Widget(value='<iframe src="http://localhost:44609/index.html?ui=P_0x7b2bb0a0c190_5&reconnect=auto" class="pyvi…